# Data From IMDb

In [1]:
!wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
!wget https://github.com/udacity/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/reviews.txt

--2019-09-01 22:50:13--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.05s   

2019-09-01 22:50:14 (4.37 MB/s) - ‘labels.txt’ saved [225000/225000]

--2019-09-01 22:50:15--  https://github.com/udacity/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/reviews.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/r

## Load in and visualize data

In [2]:
import numpy as np
with open('reviews.txt', 'r') as f:
  reviews = f.read()  
  
with open('labels.txt', 'r') as f:
  labels = f.read()

  
print(reviews[:345])
print("---")
print(labels[:10])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through th
---
positive
n


# Data Pre-Processing

Encoding each word with an integer clean-up of the data

In [3]:
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [4]:
# Removing punctuation
reviews = reviews.lower()
all_text = ''.join([char for char in reviews if char not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')

# putting all text back together. new liner is replaced by a space.
all_text = ' '.join(reviews_split)

# splitting a string using a space as a delimiter
words = all_text.split()
print(words[:30])

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me']


## Encoding the words

In [0]:
from collections import Counter
counts = Counter(words)
# counts looks like {'the': 336713, 'and': 164107, ...}

# sorts each unique word by its frequency of occurrence.
# dictionary.get('key'). If 'key' does not exist, it returns None
vocab = sorted(counts, key=counts.get, reverse=True)

# vocab looks like ['the', 'and', 'a', 'of', ...]

In [6]:
v = ['a', 'b', 'c']
r = {word: index for index, word in enumerate(v, 7)} 
print(r)

c = Counter('hello')
print(c[0])

{'a': 7, 'b': 8, 'c': 9}
0


In [7]:
# map the words to an integer starting from 1
vocab_to_int = {word: index for index, word in enumerate(vocab, start=1)}
reviews_ints = []

''' 
reviews_ints has words from reviews_split mapped to ints.
['hello word', 'hello hi world'] becomes
[[1, 2], [1, 3, 2]]
'''
for review in reviews_split:
  reviews_ints.append([vocab_to_int[word] for word in review.split()])
  
  
print(reviews_ints[0])

[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]


## Encoding Labels

1 = positive, 0 = negative

In [8]:
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])
print(encoded_labels.shape)

(25001,)


## Getting rid of zero-length reviews

In [9]:
# Outlier data
review_lens = Counter([len(x) for x in reviews_ints])
# review_lens is in the form of {length : frequency}
print(review_lens)
print(f'Zero-length reviews: {review_lens[0]}')
print(f'Maximum review length: {max(review_lens)}')

Counter({132: 185, 130: 185, 135: 178, 129: 177, 125: 177, 128: 173, 137: 171, 133: 171, 138: 170, 136: 170, 131: 170, 124: 167, 126: 167, 127: 163, 139: 162, 134: 160, 140: 156, 119: 154, 148: 151, 121: 149, 143: 148, 122: 145, 142: 143, 144: 142, 152: 142, 118: 137, 120: 136, 123: 136, 141: 134, 159: 131, 145: 131, 146: 130, 154: 129, 158: 127, 153: 126, 162: 126, 151: 126, 149: 125, 150: 124, 156: 124, 117: 123, 155: 122, 115: 122, 169: 120, 147: 120, 165: 119, 160: 116, 164: 115, 161: 113, 114: 110, 168: 109, 157: 108, 166: 107, 167: 106, 170: 102, 175: 102, 172: 102, 182: 100, 179: 98, 174: 97, 176: 96, 177: 94, 116: 94, 171: 94, 191: 94, 173: 94, 112: 93, 163: 93, 180: 92, 187: 91, 186: 90, 195: 90, 183: 89, 109: 87, 185: 86, 107: 84, 188: 83, 184: 82, 199: 82, 201: 82, 110: 80, 190: 80, 113: 80, 111: 79, 178: 78, 194: 77, 198: 76, 181: 75, 209: 73, 189: 73, 203: 73, 106: 72, 207: 72, 193: 71, 192: 71, 202: 71, 196: 69, 212: 69, 205: 69, 215: 69, 217: 69, 213: 69, 108: 68, 226: 6

In [10]:
# Remove any reviews/labels with zero length from the reviews_ints list
removal_list = []
for index, x in enumerate(reviews_ints):
  if len(x) == 0:
    print('here')
    removal_list.append(index)

print(removal_list)

for index in removal_list:
  del reviews_ints[index]
  encoded_labels = np.delete(encoded_labels, index)
  
  
  
print(f'reviews_ints size: {len(reviews_ints)}, encoded_labels size: {len(encoded_labels)}')

here
[25000]
reviews_ints size: 25000, encoded_labels size: 25000


## Padding Sequences

In [11]:
# Given a = [11, 22, 33], a[-2:] is [22, 33]
def pad_features(reviews_ints, seq_length):
  features = np.zeros((len(reviews_ints), seq_length), dtype=int)
  for i, review in enumerate(reviews_ints):
    # if the review's short, it will keep the zeros on the left and have the content on the right.
    features[i, -len(review):] = np.array(review)[:seq_length]
    
  return features
  

seq_length = 200
features = pad_features(reviews_ints, seq_length=seq_length)

assert len(features) == len(reviews_ints)
assert len(features[0] == seq_length)

#print the first 2 rows of 5 values
print(features[:2,:5])

[[0 0 0 0 0]
 [0 0 0 0 0]]


## Data Loaders and Batching

With train, validation, and test fractions equal to 0.8, 0.1, 0.1, respectively, the final, feature data shapes should look like:

Train set:          (20000, 200) 
Validation set:     (2500, 200) 
Test set:           (2500, 200)

In [12]:
import torch 
from torch.utils.data import TensorDataset, DataLoader

split_frac = 0.8

split_idx = int(len(features) * split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x) * 0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

print(train_x.shape)
print(val_x.shape)
print(test_x.shape)

print(train_y.shape)
print(val_y.shape)
print(test_y.shape)

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)


(20000, 200)
(2500, 200)
(2500, 200)
(20000,)
(2500,)
(2500,)


In [13]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,   443,  2913,  3948],
        [    0,     0,     0,  ...,  1197,  4821,   582],
        [   11,     6,     3,  ...,     5,   168,    67],
        ...,
        [13771,  3600,   907,  ...,     3,  1098, 15299],
        [   11,    18,     6,  ...,    73,   106,   620],
        [    0,     0,     0,  ...,    46,    21,    80]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 1])


# Sentiment Network in Pytorch

In [14]:
train_on_gpu=torch.cuda.is_available()

if (train_on_gpu):
  print("training on gpu")
  
else:
  print("no gpu available")

training on gpu


In [0]:
import torch.nn as nn

class SentimentRNN(nn.Module):
  
  def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, 
               n_layers, drop_prob=0.5):
    super(SentimentRNN, self).__init__()
    
    self.output_size = output_size
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim
    
    # embedding and LSTM layers
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
    
    # dropout layer
    self.dropout = nn.Dropout(0.3)
    
    # linear and sigmoid layers
    self.fc = nn.Linear(hidden_dim, output_size)
    self.sig = nn.Sigmoid()
    
    
  def forward(self, x, hidden):
    
    batch_size = x.size(0)
    
    embeds = self.embedding(x)
    lstm_out, hidden = self.lstm(embeds, hidden)
    
    # stack up lstm outputs
    lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
    
    # dropout and fully-connected layer
    out = self.dropout(lstm_out)
    out = self.fc(out)

    # sigmoid function
    sig_out = self.sig(out)

    # reshape to be batch_size first
    sig_out = sig_out.view(batch_size, -1)
    sig_out = sig_out[:, -1] # get last batch of labels

    # return last sigmoid output and hidden state
    return sig_out, hidden
  
  
  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data
    
    if (train_on_gpu):
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
    else:
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

    return hidden

In [17]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [19]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.736194... Val Loss: 0.660989
Epoch: 1/4... Step: 200... Loss: 0.628286... Val Loss: 0.644830
Epoch: 1/4... Step: 300... Loss: 0.599046... Val Loss: 0.575069
Epoch: 1/4... Step: 400... Loss: 0.542037... Val Loss: 0.522899
Epoch: 2/4... Step: 500... Loss: 0.420055... Val Loss: 0.570658
Epoch: 2/4... Step: 600... Loss: 0.483966... Val Loss: 0.516295
Epoch: 2/4... Step: 700... Loss: 0.561418... Val Loss: 0.490463
Epoch: 2/4... Step: 800... Loss: 0.304755... Val Loss: 0.471010
Epoch: 3/4... Step: 900... Loss: 0.441638... Val Loss: 0.587238
Epoch: 3/4... Step: 1000... Loss: 0.261099... Val Loss: 0.503679
Epoch: 3/4... Step: 1100... Loss: 0.381693... Val Loss: 0.449291
Epoch: 3/4... Step: 1200... Loss: 0.407934... Val Loss: 0.478880
Epoch: 4/4... Step: 1300... Loss: 0.275053... Val Loss: 0.509744
Epoch: 4/4... Step: 1400... Loss: 0.227476... Val Loss: 0.511192
Epoch: 4/4... Step: 1500... Loss: 0.162259... Val Loss: 0.519354
Epoch: 4/4... Step: 1600... Loss: 

# Testing the train model

In [21]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    # get predicted outputs
    output, h = net(inputs, h)

    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.496
Test accuracy: 0.805
